In [ ]:
'''
분산분석
    1. 일원분산분석
        1.1 ONEWAY ANOVA
        1.2 ONEWAY ANOVA - 사후검정
        
    2. 이원분산분석 
        2.1 TWOWAY ANOVA
        2.2 TWOWAY ANOVA - 사후검정
'''

In [ ]:
'''
1. 일원분산분석
        1.1 ONEWAY ANOVA

어느 제과회사는 세 가지 제빵법 A, B, C가  
제품의 부피(볼륨)에 영향을 주는지를 실험하고자 했습니다.

각 방법별로 **무작위로 5개씩** 빵을 만들어 부피를 측정했습니다:

| A제빵법 | B제빵법 | C제빵법 |
|---------|---------|---------|
| 20.1    | 22.0    | 18.5    |
| 21.0    | 21.5    | 19.0    |
| 19.5    | 22.3    | 17.5    |
| 20.3    | 23.1    | 18.2    |
| 20.7    | 21.9    | 17.9    |

이 데이터를 바탕으로,  
**세 제빵법에 따라 평균 부피에 차이가 있는지**  
**유의수준 5%에서 검정**하세요.  
(정규성, 등분산 가정)
'''

In [ ]:
# 그룹별 데이터
A_data = [20.1, 21.0, 19.5, 20.3, 20.7]
B_data = [22.0, 21.5, 22.3, 23.1, 21.9]
C_data = [18.5, 19.0, 17.5, 18.2, 17.9]

alpha = 0.05

In [ ]:
H0 = '세 제빵법에 따른 평균 부피 차이가 없다.'
H1 = '세 제빵법에 따른 평균 부피 차이가 있다.'

In [ ]:
from scipy import stats

statistic, p_value = stats.f_oneway(A_data, B_data, C_data)
print(f'statistic  :{statistic:.2f}')
print(f'p_value  :{p_value:.2f}')
if p_value < 0.05:
    print(H1)
else:
    print(H0)

In [ ]:
import numpy as np
from statsmodels.stats.oneway import anova_oneway

data = np.array(A_data + B_data + C_data).reshape(-1, 1)
groups = np.array(['A'] * len(A_data) + ['B'] * len(B_data) + ['C'] * len(C_data)).reshape(-1, 1)
anova_result = anova_oneway(data=data, groups=groups, use_var='equal')

print(f'statistic  :{anova_result.statistic:.2f}')
print(f'p_value  :{anova_result.pvalue:.2f}')
if p_value < 0.05:
    print(H1)
else:
    print(H0)

In [ ]:
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

df = pd.DataFrame({'value': A_data + B_data + C_data, 'group': ['A'] * 5 + ['B'] * 5 + ['C'] * 5})
tukey = pairwise_tukeyhsd(endog=df['value'], groups=df['group'], alpha=0.05)
print(tukey)

fig = tukey.plot_simultaneous()

In [ ]:
'''
2. 이원분산분석 
    2.1 TWOWAY ANOVA

한 스포츠 브랜드는  
운동화의 디자인(기존형, 개선형)과 운동 종류(달리기, 농구)에 따라  
착용감(Comfort Score)에 차이가 있는지를 실험하고자 했습니다.

각 조건 조합마다 5명의 참가자에게 점수를 받았고,  
데이터는 다음과 같습니다:

| 디자인  | 운동종류 | 점수 |
|--------|---------|------|
| 기존형 | 달리기 | 7.1, 6.8, 7.3, 6.9, 7.0  
| 기존형 | 농구   | 6.3, 6.5, 6.2, 6.7, 6.4  
| 개선형 | 달리기 | 7.8, 8.0, 7.5, 7.9, 7.7  
| 개선형 | 농구   | 6.9, 7.1, 7.0, 6.8, 7.2  
📌 분석 목표
디자인, 운동 종류가 착용감 점수에 영향을 주는가?
두 요인 간 상호작용 효과도 존재하는가?
'''

In [ ]:
import pandas as pd

# 데이터 구성
design = ['기존형'] * 10 + ['개선형'] * 10
exercise = ['달리기'] * 5 + ['농구'] * 5 + ['달리기'] * 5 + ['농구'] * 5
score = [
    7.1, 6.8, 7.3, 6.9, 7.0,   # 기존형-달리기
    6.3, 6.5, 6.2, 6.7, 6.4,   # 기존형-농구
    7.8, 8.0, 7.5, 7.9, 7.7,   # 개선형-달리기
    6.9, 7.1, 7.0, 6.8, 7.2    # 개선형-농구
]

# DataFrame 생성
df = pd.DataFrame({
    'design': design,
    'exercise': exercise,
    'score': score
})
df.head()

In [ ]:
from statsmodels.formula.api import ols
formula = 'score ~ C(design) + C(exercise) + C(design):C(exercise)'
model = ols(formula=formula, data=df)
model_fitted = model.fit()
model_fitted.summary()

In [ ]:
from statsmodels.stats.anova import anova_lm

anova_result = anova_lm(model_fitted, typ=2).fillna('')
anova_result

In [ ]:
if anova_result.loc['C(design)', 'PR(>F)'] < 0.05:
    print('Design은 Score에 영향을 끼친다.')
else:
    print('Design은 Score에 영향을 끼치지 않는다.')

if anova_result.loc['C(exercise)', 'PR(>F)'] < 0.05:
    print('exercise은 Score에 영향을 끼친다.')
else:
    print('exercise은 Score에 영향을 끼치지 않는다.')

if anova_result.loc['C(design):C(exercise)', 'PR(>F)'] < 0.05:
    print('design과 exercise는 서로 영향을 끼친다.')
else:
    print('design과 exercise는 서로 영향을 끼치지 않는다.')

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import pairwise_tukeyhsd

plt.rcParams['font.family'] = 'AppleGothic'
df['group'] = df['design'] + '_' + df['exercise']

tukey = pairwise_tukeyhsd(endog=df['score'], groups=df['group'], alpha=0.05)
print(tukey)

fig = tukey.plot_simultaneous()

In [ ]:
from statsmodels.stats.multicomp import MultiComparison
df['group'] = df['design'] + '_' + df['exercise']

multi_comparison = MultiComparison(data=df['score'], groups=df['group'])
multi_comparison.tukeyhsd(alpha=0.05).summary()

In [ ]:
from statsmodels.graphics.factorplots import interaction_plot
fig = interaction_plot(
    df['exercise'], df['design'], df['score'],
    )